*  for finding difference topics in the whole lot of data like suing words to find them

 *  example virat,dhoni,t20, - these words point to a topic i.e Cricket

In [ ]:
! pip install gensim nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [ ]:
from gensim.models import LdaModel,CoherenceModel
from gensim.corpora import Dictionary
import nltk,re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_data = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
documents= newsgroups_data.data

In [ ]:
def clean_text(text):
    text = re.sub(r'\W', ' ', text.lower())
    tokens = [word for word in text.split() if word not in stop_words]
    return tokens

In [ ]:
#tokenizing docs
tokenzied_docs = [clean_text(doc) for doc in documents]

In [ ]:
dictionary=Dictionary(tokenzied_docs)
dictionary.filter_extremes(no_below=10, no_above=0.9)
corpus=[dictionary.doc2bow(text) for text in tokenzied_docs]

In [ ]:
lda_model=LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=8,
    random_state=42,
     alpha='auto',
     per_word_topics=True
    )

In [ ]:
print("\nTOP WORDS PER TOPIC:\n")
for idx,topic in lda_model.show_topics(formatted=False):
    print(f"Topic: {idx+1}:{' | '.join([word[0] for word in topic])}")


TOP WORDS PER TOPIC:

Topic: 1:1 | 2 | year | 4 | 3 | one | first | last | 5 | 0
Topic: 2:one | would | like | 00 | get | know | new | good | 2 | time
Topic: 3:x | 1 | w | c | 2 | _ | n | 0 | 8 | cx
Topic: 4:would | one | people | god | think | know | like | us | say | even
Topic: 5:1 | 2 | 0 | use | 3 | windows | drive | get | edu | system
Topic: 6:ax | 0 | 1 | 3 | q | max | 2 | 7 | g | p
Topic: 7:one | space | would | use | like | also | 1 | time | may | people
Topic: 8:team | 2 | 25 | hockey | game | 3 | edu | 4 | 1 | 7


In [ ]:
coherence_model=CoherenceModel(model=lda_model,texts=tokenzied_docs,dictionary=dictionary,coherence='c_v')
coherence_score=coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score:.4f}")
#coherence score above 0.3 is good

Coherence Score: 0.5570


In [ ]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.7 MB/s eta 0:00:00


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

vis_Data=pyLDAvis.gensim_models.prepare(lda_model,corpus,dictionary,sort_topics=False)


In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_Data)